In [ ]:
import datetime
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as T
from einops import rearrange
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from src.dataset import S1GRD_MEAN, S1GRD_STD, S2L1C_MEAN, S2L1C_STD, S2L2A_MEAN, S2L2A_STD, E2SChallengeDataset
from src.submit import create_submission_from_dict, validate_submission
from torchgeo.models import RCF
from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.mps.is_available():
    device = torch.device("mps")

seed = 10012023
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


print(device)

cuda:0


### Compress

In [2]:
modalities = ["s1", "s2l1c", "s2l2a"]
mean = S1GRD_MEAN + S2L1C_MEAN + S2L2A_MEAN
std = S1GRD_STD + S2L1C_STD + S2L2A_STD

transforms = T.Compose([T.Normalize(mean=mean, std=std)])

root = "/data/data_eval/data_eval/"

dataset = E2SChallengeDataset(
    root, modalities=modalities, dataset_name="bands", transform=transforms, concat=True, output_file_name=True
)
len(dataset)

8111

In [5]:
dataset[0].keys(), dataset[0]["data"].shape, dataset[0]["file_name"]

(dict_keys(['data', 'file_name']),
 torch.Size([1, 4, 27, 264, 264]),
 '8f3287a462a96da37b5b7e2e2a92dd94a28123251fa81a4b549f25079d0b7460')

In [6]:
# need to create custom dataset for RCF to perform ZCA whitening on the dataset
class MOSAIKSE2SDataset(E2SChallengeDataset):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __getitem__(self, idx):
        sample = super().__getitem__(idx)
        t = torch.randint(low=0, high=4, size=(1,)).item()
        sample = sample[0, t]
        return dict(image=sample)


empirical_dataset = MOSAIKSE2SDataset(
    root, modalities=modalities, dataset_name="bands", transform=transforms, concat=True, output_file_name=False
)
len(empirical_dataset)

8111

In [7]:
empirical_dataset[0]["image"].shape

torch.Size([27, 264, 264])

In [8]:
model = RCF(
    in_channels=27, features=4096, kernel_size=3, bias=-1.0, seed=seed, mode="empirical", dataset=empirical_dataset
)
model.eval()
model = model.to(device)

In [9]:
tmp_dir = "tmp_embeddings_10012023"
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

with torch.inference_mode():
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        x = sample["data"]
        b, t, c, h, w = x.shape
        x = rearrange(x, "b t c h w -> (b t) c h w")
        x = x.to(device)
        emb = model(x)
        emb = rearrange(emb, "(b t) c -> b t c", b=b, t=t)
        emb = emb.detach().cpu().numpy()

        np.save(os.path.join(tmp_dir, f"{idx}.npy"), emb)
        filenames.append(sample["file_name"])

# Load and concatenate after loop (safer)
embeddings = [np.load(os.path.join(tmp_dir, f"{i}.npy")) for i in range(len(filenames))]
embeddings = np.concatenate(embeddings, axis=0)

# Save in same format as original code
np.savez("mosaik-embeddings_4096_3x3_seed_10012023.npz", embeddings=embeddings, filenames=filenames)

100%|██████████| 8111/8111 [19:48<00:00,  6.82it/s]


In [10]:
embeddings.shape, len(filenames)

((8111, 4, 4096), 8111)

output = f"mosaik-embeddings_4096_3x3_seed_10012023.npz"
data = dict(embeddings=embeddings, filenames=filenames)
np.savez(output, **data)

### Load the MOSAIKS-ed embedding

#### Test Embedding

In [11]:
data_test = np.load("mosaik-embeddings_4096_3x3_seed_10012023.npz")

embeddings_test = data_test["embeddings"]
filenames_test = data_test["filenames"]

embeddings_test.shape, embeddings_test.dtype, len(filenames_test)

((8111, 4, 4096), dtype('float32'), 8111)

Mean pooling over temporal dim

In [14]:
embeddings_test = np.mean(embeddings_test, axis=1)

embeddings_test.shape, f"Rank={int(np.linalg.matrix_rank(embeddings_test))}"

((8111, 4096), 'Rank=372')

#### Dev Embedding -- but test for now -- only to see if the embeddings are the same

In [ ]:
data_dev = np.load(
    "mosaik-embeddings_4096_3x3_seed0_test_submission_justtobesure_v3_seed0sameasv1and2_butalsoglobalseeded.npz"
)
# data_dev = np.load(f"mosaik-embeddings_4096.npz")

embeddings_dev = data_dev["embeddings"]
filenames_dev = data_dev["filenames"]

embeddings_dev.shape, embeddings_dev.dtype, len(filenames_dev)

In [ ]:
embeddings_dev = np.mean(embeddings_dev, axis=1)

embeddings_dev.shape, f"Rank={int(np.linalg.matrix_rank(embeddings_dev))}"

In [ ]:
# Fit PCA on dev
pca = PCA(n_components=1024, random_state=42)
embeddings_dev_pca = pca.fit_transform(embeddings_dev)

# Apply PCA to test
embeddings_test_pca = pca.transform(embeddings_test)

# Explained variance
explained_var_dev = np.sum(pca.explained_variance_ratio_)
explained_var_test = np.var(embeddings_test_pca) / np.var(embeddings_test)
print(f"Explained Variance - Dev:  {explained_var_dev:.4f}")
print(f"Explained Variance - Test: {explained_var_test:.4f}")

# Check first 5 PCA component stats
print("\nFirst 5 PCA components mean/var:")
for i in range(5):
    d_mean, d_var = embeddings_dev_pca[:, i].mean(), embeddings_dev_pca[:, i].var()
    t_mean, t_var = embeddings_test_pca[:, i].mean(), embeddings_test_pca[:, i].var()
    print(f"PC{i + 1}: Dev {d_mean:.4f}/{d_var:.4f} | Test {t_mean:.4f}/{t_var:.4f}")

# Optional: t-SNE visualization
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
combined = np.vstack([embeddings_dev_pca, embeddings_test_pca])
tsne_proj = tsne.fit_transform(combined)
tsne_dev = tsne_proj[: len(embeddings_dev_pca)]
tsne_test = tsne_proj[len(embeddings_dev_pca) :]

plt.figure(figsize=(8, 6))
plt.scatter(tsne_dev[:, 0], tsne_dev[:, 1], s=5, label="Dev", alpha=0.5)
plt.scatter(tsne_test[:, 0], tsne_test[:, 1], s=5, label="Test", alpha=0.5)
plt.legend()
plt.title("t-SNE of PCA outputs (Dev vs Test)")
plt.show()

Explained Variance for both embeddings

In [ ]:
# Absolute PCA scores
abs_test_scores = np.abs(embeddings_test_pca)
abs_dev_scores = np.abs(embeddings_dev_pca)

# Top contributing PC for each sample
test_top_pc = np.argmax(abs_test_scores, axis=1)
dev_top_pc = np.argmax(abs_dev_scores, axis=1)

# Plot histograms of top contributing PCs
plt.hist(test_top_pc, bins=50, alpha=0.6, label="Test", density=True)
plt.hist(dev_top_pc, bins=50, alpha=0.6, label="Dev", density=True)
plt.xlabel("Top contributing PC index")
plt.ylabel("Density")
plt.title("Most Influential PC per Embedding")
plt.legend()
plt.grid()
plt.show()

In [ ]:
test_ranks = np.argsort(-abs_test_scores, axis=1)[:5]
dev_ranks = np.argsort(-abs_dev_scores, axis=1)[:5]

print("Test Embeddings - Top 5 PCs by contribution:")
print(test_ranks)

print("\nDev Embeddings - Top 5 PCs by contribution:")
print(dev_ranks)

In [ ]:
pc_var_test = embeddings_test_pca.var(axis=0)
pc_var_dev = embeddings_dev_pca.var(axis=0)

plt.plot(pc_var_test[:50], label="Test")
plt.plot(pc_var_dev[:50], label="Dev")
plt.xlabel("PC index")
plt.ylabel("Variance")
plt.title("Variance per PC (First 50 PCs)")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np


def kl_divergence_gaussian(mu_p, std_p, mu_q, std_q):
    return np.log(std_q / std_p) + (std_p**2 + (mu_p - mu_q) ** 2) / (2 * std_q**2) - 0.5


# Compute means and stds per PC
mu_test = embeddings_test_pca.mean(axis=0)
std_test = embeddings_test_pca.std(axis=0)
mu_dev = embeddings_dev_pca.mean(axis=0)
std_dev = embeddings_dev_pca.std(axis=0)

# Add small epsilon to avoid division by zero
eps = 1e-6
std_test = np.maximum(std_test, eps)
std_dev = np.maximum(std_dev, eps)

# KL divergence for each PC
kl_per_pc = kl_divergence_gaussian(mu_test, std_test, mu_dev, std_dev)

# Plot top PCs with highest KL
top_k = 50
sorted_idx = np.argsort(-kl_per_pc[:top_k])

plt.bar(range(top_k), kl_per_pc[sorted_idx])
plt.xticks(range(top_k), sorted_idx, rotation=90)
plt.xlabel("PC Index")
plt.ylabel("KL Divergence")
plt.title(f"KL Divergence per PC (Top {top_k})")
plt.grid()
plt.tight_layout()
plt.show()

# Print top PCs with highest KL
print("Top 10 PCs with highest KL divergence:")
for i in sorted_idx[:10]:
    print(f"PC{i}: KL = {kl_per_pc[i]:.5f}")

In [ ]:
# Combined scatter plot of PC1 vs PC2
plt.figure(figsize=(6, 5))
plt.scatter(embeddings_test_pca[:, 0], embeddings_test_pca[:, 1], s=5, alpha=0.5, label="Test", color="blue")
plt.scatter(embeddings_dev_pca[:, 0], embeddings_dev_pca[:, 1], s=5, alpha=0.5, label="Dev", color="orange")
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.title("PC1 vs PC2: Test vs Dev")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Explained variance comparison
# Compute PCA separately on each set to compare their own variance structure
pca_test = PCA(n_components=50)
pca_dev = PCA(n_components=50)

pca_test.fit(embeddings_test)
pca_dev.fit(embeddings_dev)

var_test = pca_test.explained_variance_ratio_
var_dev = pca_dev.explained_variance_ratio_

# Plot
plt.figure(figsize=(7, 5))
plt.plot(var_test, label="Test")
plt.plot(var_dev, label="Dev")
plt.xlabel("PC Index")
plt.ylabel("Explained Variance Ratio")
plt.title("Explained Variance per PC (First 50 PCs)")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

#### Go ahead with submission

PCA 4096 --> 1024

In [15]:
filenames = filenames_test
embeddings = embeddings_test

In [16]:
pca = PCA(n_components=1024, random_state=seed)
compressed = pca.fit_transform(embeddings)
embeddings = compressed
embeddings.shape

(8111, 1024)

### Create submission

In [17]:
embedding_dict = {fname: emb for fname, emb in zip(filenames, embeddings, strict=False)}
submission_file = create_submission_from_dict(embedding_dict)
print("Number of embeddings:", len(submission_file))
submission_file.head()

Number of embeddings: 8111


,id,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,8f3287a462a96da37b5b7e2e2a92dd94a28123251fa81a...,-214.474792,24.645157,245.912201,-35.895252,55.638908,-178.627808,-141.998062,-4.258297,-7.816849,...,-0.160565,0.378877,0.008854,-0.101823,-0.071167,0.129678,-0.239169,-0.009011,-0.009456,0.077354
1,656de524710233dbe0360f1317de7ea0551779599c33d8...,-307.475311,-116.031952,-5.405784,68.430809,-32.713799,-401.357605,202.659866,-157.131638,90.226646,...,0.061839,-0.093697,0.071291,0.100546,0.102545,-0.271798,-0.160761,-0.081420,0.005467,0.035779
2,6d3ec7e953f307f741d335fcf6d71667ed11ae87e16f59...,-333.217194,-270.502472,28.059067,-64.826454,57.852196,-70.070358,-48.072166,117.565437,63.463100,...,0.020282,-0.080376,-0.056329,0.061448,-0.091161,-0.171986,0.172369,-0.078218,0.154074,0.050406
3,806bd4494d4b8cb3ffb82dfbfe7c755f314723ccd3194c...,99.931389,91.669434,-7.907872,337.314789,15.303823,-253.382416,-297.459961,118.035797,124.275223,...,-0.108617,-0.398720,0.386885,-0.217956,-0.053034,0.294437,0.255101,-0.262474,-0.748182,0.199978
4,57f76ad8c83649d1367d04387da42ae9f3529047c8407d...,3910.951172,-361.237610,27.700787,-175.410797,-579.450439,233.798370,-117.907013,-5.358448,-37.172401,...,-0.074196,-0.157307,-0.008545,-0.656596,-0.721653,-0.426497,-0.437804,1.263510,-1.358840,-0.027554


In [18]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
path_to_output_file = f"./submission_{timestamp}.csv"
# path_to_output_file = f"/data/submission_{timestamp}.csv"
submission_file.to_csv(path_to_output_file, index=False)
embedding_ids = set(embedding_dict.keys())
embedding_dim = 1024
assert validate_submission(path_to_output_file, embedding_ids, embedding_dim)